In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt

In [ ]:
import nltk 
import nltk.corpus
from nltk.corpus import stopwords
from nltk.tokenize import BlanklineTokenizer
from nltk.tokenize import TweetTokenizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
import string

In [ ]:
import re
from nltk.tokenize import word_tokenize
import gensim
from keras.preprocessing.text import Tokenizer

In [ ]:
from nltk.stem import PorterStemmer 
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/kaggle/input/twitter-airline-sentiment/Tweets.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.airline_sentiment.value_counts()

In [ ]:
#notre objectif est de prédicter la valeur de la colonne "airlaine_sentiment"

In [ ]:
df.info()

In [ ]:
# les colonnes "airline_sentiment_gold " et "negativereason_gold" ont trop peu de valeurs non nulles
#donc on va les supprimer

In [ ]:
df.drop(['airline_sentiment_gold','negativereason_gold'],axis=1,inplace=True)

In [ ]:
df.tweet_coord.nunique()

In [ ]:
#parmi 1019 valeurs non nulles du colonne 'tweet_coord' ilya 832 valeurs uniques
#donc c'est une colonne peu important

In [ ]:
df.drop(['tweet_coord'],axis=1,inplace=True)

In [ ]:
df.info()

In [ ]:
df.negativereason.value_counts()

In [ ]:
df.negativereason.nunique()

In [ ]:
#trés bien il ya que 10 valeurs uniques

In [ ]:
nr = df.negativereason.unique()
nr

In [ ]:
nr.reshape(1,11)

In [ ]:
nr[0]

In [ ]:
nr[5]

In [ ]:
df.negativereason.iloc[0] = 'Bad Flight'

In [ ]:
df.negativereason.iloc[0]

In [ ]:
df.negativereason.iloc[1] is np.NaN

In [ ]:
import random
for k in range (df.shape[0]):
  if df.negativereason.iloc[k] is np.NaN:
    p = random.randint(1, 10)
    df.negativereason.iloc[k] = nr[p]

In [ ]:
df.info()

In [ ]:
#on va vérifier le résultat
df.negativereason.value_counts()

In [ ]:
#trés bien

In [ ]:
df.negativereason_confidence.nunique()

In [ ]:
#1410 valeurs se répétent dans 10522 lignes donc on va les utiliser pour remplacer les valeurs 'NaN'
nr = df.negativereason_confidence.unique()
len(nr)

In [ ]:
nr.reshape(1,1411)

In [ ]:
df.negativereason_confidence = df.negativereason_confidence.astype('object')

In [ ]:
df.info()

In [ ]:
for k in range (df.shape[0]):
  if df.negativereason_confidence.iloc[k] is np.NaN:
    p = random.randint(1, 1411)
    df.negativereason_confidence.iloc[k] = nr[p]

In [ ]:
df.info()

In [ ]:
df.user_timezone.nunique()

In [ ]:
#trés bien juste 85 uniques valeurs qui se répétent dans plus de 9000 lignes

In [ ]:
nr = df.user_timezone.unique()
nr.reshape(1,86)

In [ ]:
for k in range (df.shape[0]):
  if df.user_timezone.iloc[k] is np.NaN:
    p = random.randint(1, 85)
    df.user_timezone.iloc[k] = nr[p]

In [ ]:
df.info()

In [ ]:
df.tweet_location.nunique()

In [ ]:
nr = df.tweet_location.unique()
nr.shape

In [ ]:
nr.reshape(1,3082)

In [ ]:
for k in range (df.shape[0]):
  if df.tweet_location.iloc[k] is np.NaN:
    p = random.randint(1, 3081)
    df.tweet_location.iloc[k] = nr[p]

In [ ]:
df.info()

In [ ]:
#ilya un probléme avec la colonne 'negativereason_confidence'

In [ ]:
df.head()

In [ ]:
df.negativereason_confidence.iloc[0]  in range(0,2)

In [ ]:
df.negativereason_confidence.iloc[0] = 1

In [ ]:
df.negativereason_confidence.iloc[0]  

In [ ]:
nr = df.negativereason_confidence.unique()

In [ ]:
nr.shape

In [ ]:
nr.reshape(1,1411)

In [ ]:
nr[2] = 0.644

In [ ]:
nr

In [ ]:
for k in range (df.shape[0]):
  if df.negativereason_confidence.iloc[k] not in range(0,2):
    p = random.randint(0,1410)
    df.negativereason_confidence.iloc[k] = nr[p]

In [ ]:
df.info()

In [ ]:
#trés bien 

In [ ]:
df.head()

In [ ]:
df.drop(['tweet_id'],axis=1,inplace=True)

In [ ]:
df.name.nunique()

In [ ]:
X = df.drop(['airline_sentiment'],axis=1)
y = df.airline_sentiment

In [ ]:
y = y.replace({'negative' : 0 , 'neutral' : 1 , 'positive' : 2})

In [ ]:
#on va traiter la colonne "tweet_created"

In [ ]:
X['year'] = pd.to_datetime(X.tweet_created).dt.year
X['day'] = pd.to_datetime(X.tweet_created).dt.day
X['month'] = pd.to_datetime(X.tweet_created).dt.month
X['hour'] = pd.to_datetime(X.tweet_created).dt.hour
X['minute'] = pd.to_datetime(X.tweet_created).dt.minute
X['second'] = pd.to_datetime(X.tweet_created).dt.second

In [ ]:
X.head()

In [ ]:
X.drop(['tweet_created'],axis=1,inplace=True)

In [ ]:
X.drop(['name'],axis=1,inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
X.negativereason = X.negativereason.astype('str')
X.airline = X.airline.astype('str')
X.tweet_location = X.tweet_location.astype('str')
X.user_timezone = X.user_timezone.astype('str')
columns_to_Encode = ["negativereason","airline","tweet_location","user_timezone"]
le = LabelEncoder()
for each in columns_to_Encode:
    X[each] = le.fit_transform(X[each])

In [ ]:
X.head()

In [ ]:
#Etape suivante : traiter la colonne "text"

In [ ]:
X.tail()

In [ ]:
X['len_text'] = X['text'].apply(len)

In [ ]:
plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(10,5))
sns.distplot(X['len_text'],kde=False,color='red',hist=True)
plt.xlabel("text Length",size=15)
plt.ylabel("Frequency",size=15)
plt.title("Length Histogram",size=15)

In [ ]:
from nltk.stem.porter import PorterStemmer

In [ ]:
ps = PorterStemmer() #pour "text preprocessing"

message = []

for i in range(0, X.shape[0]):
    #accepter que les mots alphabétiques
    review = re.sub('[^a-zA-Z]', ' ', X['text'][i])
    #convertir tous minuscule
    review = review.lower()
    #splitter chaque ligne
    review = review.split()
    #
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    #construire de nouveau la ligne
    review = ' '.join(review)
    message.append(review)

In [ ]:
X['clean_text']=np.empty((len(message),1))
for i in range(len(message)):
    X['clean_text'][i]=message[i]
X['clean_text_len']=X['clean_text'].apply(len)
X.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=15000)
X1 = cv.fit_transform(message).toarray()

In [ ]:
X1.shape

In [ ]:
X2 = X.copy()

In [ ]:
X2.drop(['text','clean_text'],axis=1,inplace=True)

In [ ]:
X3 = pd.DataFrame(data=X1)

In [ ]:
X3.shape

In [ ]:
X4 = pd.concat([X2,X3],axis=1)

In [ ]:
X4.shape

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X4,y,test_size=0.2,random_state=42)


In [ ]:
#modéle1

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier(n_estimators=100,criterion='gini',random_state=42)

In [ ]:
rfc.fit(x_train,y_train)

In [ ]:
y_pred = rfc.predict(x_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbk = GradientBoostingClassifier(random_state=42, n_estimators=100,min_samples_split=100)
gbk.fit(x_train, y_train)
gbk_predict = gbk.predict(x_test)
print("Gradient Boosting Score :",accuracy_score(y_test,gbk_predict ))

In [ ]:
#82% précision c'est bien
#on peut essayer avec un neural network pour obtenir plus de précision